In [2]:
from numpy import array, append, average, cov, newaxis, round, r_
from numpy.linalg import inv
from pandas import DataFrame, Series

In [3]:
# AGGREGATION#
############### inputs (you can change them) ###############
h = array([[2000000, 800000]]).T  # holdings
v_h = 70000000  # budget 
pi = array([[-0.43, -0.19, 0.25, 0.31], 
            [0.15, -1.63, -0.05, 0.91]])  # joint scenarios
p = array([[0.3, 0.1, 0.4, 0.2]])  # probabilities
v = array([[16., 47.5]]).T
############################################################

r = pi/v  # instrument return scenarios
print('r \n', r.round(4))

w = h*v/v_h  # weights
print('w \n', w.round(2))

r_w = w.T@r  # portfolio return scenarios
print('r_w =', r_w.round(4))
print('p =', p)

############# inputs (you can change them) ##############
h_b = array([[1000000, 1200000]]).T  # benchmark holdings
v_b = 73000000  # benchmark budget
#########################################################

w_b = h_b*v/v_b  # benchmark weights
print('w_b \n', w_b.round(2))

r_b = w_b.T@r  # benchmark return scenarios
print('r_b =', r_b.round(4))

r_ex_w = r_w - r_b  # excess return
print('r_ex_w =', r_ex_w.round(4))
print('p =', p)

output = DataFrame({'j_bar': Series(p.shape[1]), 'n_bar': Series(pi.shape[0]), 'r_excess': Series(r_ex_w.flatten()), 
                    'r_w': Series(r_w.flatten()), 'pi': Series(pi.reshape(-1)), 'p': Series(p.flatten()), 'r': Series(r.flatten()), 
                    'r_b': Series(r_b.flatten()), 'h': Series(h.flatten()), 'h_b': Series(h_b.flatten()), 'v_h': Series(v_h),
                    'v_b': Series(v_b), 'w': Series(w.flatten()), 'w_b': Series(w_b.flatten())})
output.to_csv('~/Databases/temporary-databases/db_aggregation_scenario_numerical.csv')


r 
 [[-0.0269 -0.0119  0.0156  0.0194]
 [ 0.0032 -0.0343 -0.0011  0.0192]]
w 
 [[0.46]
 [0.54]]
r_w = [[-0.0106 -0.0241  0.0066  0.0193]]
p = [[0.3 0.1 0.4 0.2]]
w_b 
 [[0.22]
 [0.78]]
r_b = [[-0.0034 -0.0294  0.0026  0.0192]]
r_ex_w = [[-0.0071  0.0053  0.004   0.0001]]
p = [[0.3 0.1 0.4 0.2]]


In [4]:
## TOP-DOWN FACTOR ON DEMAND ##
################## inputs (you can change them) ##################
rh_z = array([[-0.01057143, -0.0041252, -0.01986819],
              [-0.02405714, -0.00980853, 0.01450357],
              [0.00657143, -0.00406089, 0.01188747],
              [0.01925714, 0.02680999, 0.00541017]])  # scenarios
p = array([0.3, 0.1, 0.4, 0.2])  # probabilities
##################################################################

e_rh_z = average(rh_z, weights=p, axis=0)  # joint mean of return and risk factors
cv_rh_z = cov(rh_z.T, aweights=p, bias=True)  # joint covariance of return and risk factors
print('cv_rh_z \n', cv_rh_z.round(5))

beta = cv_rh_z[0, 1:]@inv(cv_rh_z[1:, 1:])  # top-down exposures
print('beta =', beta.round(2))

alpha = e_rh_z[0] - beta@e_rh_z[1:]  # shift term
print('alpha =', alpha.round(4))

out = DataFrame({'p': Series(p),'beta': Series(beta), 'alpha': Series(alpha),
                 'rh_z': Series(rh_z.reshape((p.shape[0]*3,)))})
out.to_csv('~/Databases/temporary-databases/db_attr_factors.csv', index=None)

cv_rh_z 
 [[1.8e-04 1.3e-04 8.0e-05]
 [1.3e-04 1.6e-04 2.0e-05]
 [8.0e-05 2.0e-05 2.0e-04]]
beta = [0.75 0.33]
alpha = -0.0007


In [7]:
##JOINT DISTRIBUTION OF RESIDUAL##

db_attr_factors = read_csv('~/Databases/temporary-databases/db_attr_factors.csv')
p = array(db_attr_factors['p'][:4])  # probabilities
rh_z = db_attr_factors['rh_z'].to_numpy().reshape((-1, 3))  # scenarios
beta = array(db_attr_factors['beta'][:2])  # top-down exposures
alpha = db_attr_factors['alpha'][0]  # shift term

u = rh_z[:, 0] - alpha - beta@rh_z[:, 1:].T  # residual scenarios
u_z1_z2 = (r_['-1', u[newaxis, ...].T, rh_z[:, 1:3]]).T  # joint scenarios of residual and factors
print('u_z1_z2 \n', u_z1_z2.round(4))
print('p =', p)

beta_0 = 1  # exposure to residual
beta_factors_only = append(beta_0, beta)  # updated exposures
print('beta_factors_only =', beta_factors_only.round(2))

z0 = alpha + u  # scenarios for risk factor Z0
z0_z1_z2 = r_[z0.reshape(1, -1), u_z1_z2[1:, :]]  # risk factors with no shift
print('z0_z1_z2 \n', z0_z1_z2.round(4))
print('p =', p)

out = DataFrame({'k_': Series(2), 'j_': Series(p.shape[0]),'p': Series(p),'beta': Series(beta), 'alpha': Series(alpha),
                 'rh_z': Series(rh_z.reshape((p.shape[0]*3,))), 'uz': Series(u_z1_z2.reshape((p.shape[0]*3,)))})
out.to_csv('~/Databases/temporary-databases/db_attribution_scen_prob.csv', index=None)

NameError: name 'read_csv' is not defined

In [6]:
##  RISK ATTRIBUTION ##

db_attribution_scen_prob = read_csv('~/Databases/temporary-databases/db_attribution_scen_prob.csv')
j_bar = int(db_attribution_scen_prob['j_'].iloc[0])  # number of Monte Carlo simulations
k_bar = int(db_attribution_scen_prob['k_'].iloc[0])  # number of stocks
p = array(db_attribution_scen_prob['p'].iloc[:j_bar])  # probabilities
alpha = array(db_attribution_scen_prob['alpha'].iloc[0])  # shift term
beta = array(db_attribution_scen_prob['beta'].iloc[:j_bar-2]).reshape(k_bar, 1)  # top-down exposures
# scenario realizations of ex-ante performance and factors
rz = array(db_attribution_scen_prob['rh_z'].iloc[:j_bar*(k_bar + 1)]).reshape((j_bar, k_bar + 1))
# scenario realizations of residual and factors
uz = array(db_attribution_scen_prob['uz'].iloc[:j_bar*(k_bar + 1)]).reshape((k_bar + 1), j_bar).T

beta_0 = 1  # exposure to residual
beta = append(beta_0, beta)  # updated exposures
u = uz[:, 0]  # residual scenarios
z_0 = alpha + u  # factor scenarios
z = r_['-1', z_0.reshape(j_bar, 1), uz[:, 1:]]  # updated risk factor scenarios

sigma2_z = ((z - p@z).T*p)@(z - p@z)  # covariance of factors

r = rz[:, 0]  # return scenarios
var_r = ((r - p@r).T*p)@(r - p@r)  # variance of portfolio return
sd_r = sqrt(var_r)  # standard deviation of portfolio return

satis_r = -sd_r  # satisfaction is negative standard deviation

sd_bz = -abs(beta)*sqrt(diag(sigma2_z)).T  # non-normalized "first in" contributions
print('sd_bz =', sd_bz.round(4))

gamma_isol = satis_r/sum(sd_bz)  # normalization constant
print('gamma_isol =', gamma_isol.round(2))

sd_isol = gamma_isol*sd_bz  # "first in" proportional contributions
print('sd_isol =', sd_isol.round(4))

# non-normalized "last in" contributions
satis_diff = satis_r + sqrt(satis_r**2 + (beta*beta)*diag(sigma2_z).T - 2*beta*(beta@sigma2_z))
print('satis_diff =', satis_diff.round(4))

gamma_last = satis_r/sum(satis_diff)  # normalization constant
print('gamma_last =', gamma_last.round(2))

sd_last = gamma_last*satis_diff  # "last in" proportional contributions
print('sd_last =', sd_last.round(4))

NameError: name 'read_csv' is not defined